# YOUTUBE crawler

Coleta de comentários de vídeos em canais do youtube

In [1]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from time import sleep
import re


In [2]:
def get_url_video(url_channel):
    urls = []
    video_titles = []

    with Chrome('/Users/bruno/chromedriver') as driver:
        wait = WebDriverWait(driver,50)
        driver.get(url_channel)

        for item in range(10): 
            wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
            time.sleep(15)


        for video_title in wait.until(EC.presence_of_all_elements_located((By.ID, "video-title"))):
            video_titles.append(video_title.text)
            print(f'Título do vídeo: {video_title.text}')
            
        
        for url in driver.find_elements_by_css_selector("#video-title"):
            urls.append(url.get_attribute('href'))
            print(f'URL: {url.get_attribute("href")}')      
        
        df = pd.DataFrame(video_titles, columns=['video_title'])
        df['url'] = urls   
          

    return (df)


In [3]:
def get_video_comment(video):
    comments=[]
    authors = []

    with Chrome('/Users/bruno/chromedriver') as driver:
        wait = WebDriverWait(driver,50)
        driver.get(video)

        count = 0 
        for item in range(5): 
            wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
            if count == 0:
                driver.execute_script("window.scrollTo(0, 400);")
                time.sleep(30)
            time.sleep(30)
            count= count+1
        
        
        for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))):
            comments.append(comment.text)
            print(f'Comentário: {comment.text}')

        
        for author in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#author-text"))):
            authors.append(author.text)
            print(f'Autor: {author.text}')

        df = pd.DataFrame(comments, columns=['comment'])
        df['author'] = authors


    return df

In [4]:
def video_list(df):
    
    videos_title = df.to_list()
    lista =[]

    for title in videos_title:
        lista.append(f'coments_of_{title}')
    
    return lista

In [5]:
def concat_df(lista, folder):
    print('Função data_full')
    print(lista)
    print(type(lista))
    dfs = []
    
    count = 0 
    for l in lista[:5]:
        dataframe = pd.read_csv(f'/Users/bruno/{folder}/coments_of_{count}.csv')
        dataframe['video_title'] = l
        dfs.append(dataframe)
        count = count+1
        print(count)
    
    df = pd.concat(dfs, ignore_index=True)

    return df

In [6]:
def youtube_crawler(url_channel, folder):

    print(f'ETAPA 01 - Processando canal: {folder}')
    df_youtube_videos = get_url_video(url_channel)
    
    print(f'ETAPA 02 - Coletando vídeos do canal')
    print(f'Dataframe de vídeos: {df_youtube_videos.head()}')
    df_youtube_videos.to_csv(f'/Users/bruno/{folder}/youtube_videos.csv', index=False)
    
    print('ETAPA 03 - Coletando vídeos já processados')
    df_videos_processados = pd.read_csv(f'/Users/bruno/{folder}/youtube_videos.csv')
    lista_videos_processados = df_videos_processados.video_title.tolist()

    print('ETAPA 04 - Montando lista de vídeos para serem processados')
    youtube_video_list = df_youtube_videos.url.tolist()
    #youtube_video_list = youtube_video_list - lista_videos_processados
    name_video_list = df_youtube_videos.video_title.tolist()
    videos_processados = []
    print('ETAPA 05 - Coleta dos comentários dos vídeos listados')
    count = 0 
    for youtube_video in youtube_video_list[:5]:

        print(f'{count} - vídeo: {youtube_video}')
        df_coments = get_video_comment(video=youtube_video)
        #df_coments.to_csv(f'/Users/bruno/{folder}/coments_of_{name_video_list[count]}.csv', index=False)
        df_coments.to_csv(f'/Users/bruno/{folder}/coments_of_{count}.csv', index=False)
        count = count+1
        videos_processados.append(youtube_video)
        df_videos_processados = pd.DataFrame(videos_processados, columns=['videos_processados'])
        df_videos_processados.to_csv(f'/Users/bruno/{folder}/videos_processados.csv', index=False)

    print('ETAPA 06 - Montagaem dataframe full')
    dataframe_youtube_video = pd.read_csv(f'/Users/bruno/{folder}/youtube_videos.csv')
     
    lista = video_list(dataframe_youtube_video['video_title'])
    data = concat_df(lista, folder)
    data.to_csv(f'/Users/bruno/{folder}/data_full_{folder}.csv')
    return data

In [7]:
url_channel = 'https://www.youtube.com/channel/UCQnmd3WsYwXOz_1Z5BmAajg'
#url_channel = 'https://www.youtube.com/channel/UCjnsqwu4pnFtRg6khlxAgyw'

folder = 'Escola de Costurar'
youtube_crawler(url_channel,folder)

ETAPA 01 - Processando canal: Escola de Costurar
Título do vídeo: Empreendedorismo com especialistas do SEBRAE - A Voz da Costura :: AO VIVO
Título do vídeo: Veja como a CMC Jaciara mudou de vida com a costura
Título do vídeo: [LIVE 1] Costurando um Vestido de Manga Godê AO VIVO | Semana CMC
Título do vídeo: [AULA 1] As Novas Costureiras de Mão Cheia do Brasil | Semana CMC
Título do vídeo: 
Título do vídeo: 
Título do vídeo: Como imprimir molde em tamanho real - PDF
Título do vídeo: Vestido sem molde MUITO FÁCIL ::: Ao Vivo 24.01.19
Título do vídeo: Calcinha De Renda Passo a Passo #especial50mil
Título do vídeo: 6 dicas IMPERDÍVEIS de costura ::: Especial para iniciantes
Título do vídeo: 
Título do vídeo: 
Título do vídeo: Empreendedorismo com especialistas do SEBRAE - A Voz da Costura :: AO VIVO
Título do vídeo: [LIVE 1] Costurando um Vestido de Manga Godê AO VIVO | Semana CMC
Título do vídeo: Vai, Paulo! - Calça social - Parte 03 - Bolso traseiro e laterais
Título do vídeo: Lista de 

Autor: Neusa Rodrigues
Autor: maria de nasare Nunes
Autor: Benedita Oliveira
Autor: Maria Dajuda
Autor: Vera clair De moura
Autor: Francisca Luiza Camelo
Autor: Rosely Scarabeli Diniz
Autor: FERNANDA ARTESANATOS ENSINA
Autor: Maria Pontes
Autor: Maria Pontes
Autor: Janileia Jesus
Autor: Flor de Maria Pires Mendes
Autor: maria de nasare Nunes
Autor: ivanete moreno manfre
Autor: Terezinha Ferreira dos Santos
Autor: Regina Pereira
Autor: Maria Dajuda
Autor: Benedita Oliveira
Autor: Flor de Maria Pires Mendes
Autor: Rosenir Abreu Abreu
Autor: cicera maria
Autor: Maria Pontes
Autor: fox game 378 henrique
Autor: Maria Dajuda
Autor: Maria Dajuda
Autor: Rosenir Abreu Abreu
Autor: Maria Dajuda
Autor: Glacia Costa
Autor: Marina Felipe
Autor: Rita Ferreira
Autor: Jureni Dos Santos
Autor: Maria Dajuda
Autor: Jureni Dos Santos
Autor: Teresinha Fraga
Autor: Dorinha Goncalves
Autor: Carmem Lucia
Autor: Edirlei Silva
Autor: Nenem Vassorosa
Autor: Mariusa Fernandes
Autor: Marina Felipe
Autor: Neya Mart

Comentário: Marizete f Santana na.vai encimar overloque também
Comentário: Eu sou essa Maria, dependo de meu marido pra tudo😞
Comentário: Eu assisti, perdi a live por motivos pessoais de família, mas vou assistir atrasada,vou tentar não perder,amo costurar,te agradeço muito, abraço.
Comentário: Eu também realmente sou essa Maria.
Comentário: Já costurei quase todo, falta o viés do meio e a bainha
Comentário: Eu assisti e fiz o vestido,com tecido que eu tinha em casa.
Comentário: SOU DE CAMPINAS SÃO PAULO, ONTEM FIQUEI NO HOSPITAL DA PUC E PERDI A LIVE, CONSEGUI COMPRAR O MATERIAL...
Comentário: Eu ja costuro so quero aprender novas tecnicas  to achando muito boa, a outra coisa, quero saber Como vejo se estou nas normas, legais  ? Pra participar do sorteios.
Comentário: Sou Janete  Zagatti, de Serrana interior de São Paulo. Estou amando as aulas!
Comentário: Nunca costurei será que por eu não ter prática?
Comentário: Semana CMC, vou compartilhar também.
Comentário: Dá um alô aqui para n

Comentário: Eu sua uma Maria com certeza aos 46 anos 😍
Comentário: Gostei bastante, vou costurar, vou fazer um pra mim...
Comentário: Sou Maria do Socorro de Laranjal do JARI. AP
Comentário: Perdir a aula de ontem, quero saber se eu ainda participo das aulas ou estou fora
Comentário: EU SOU A DIVINA MORAIS DIAS DE UBERLÃNDIA MG. EU assiste a aula
 adorei.
Comentário: Amei, o modelo.
Sou curiosa em costura, não gosto do meu acabamento e adorei as dicas ...GRATIDÃO
Comentário: Sou de Taboão da Serra SP
Comentário: Já estou na segunda aula, é estou gostando muito.
Comentário: Ivanilde valeria burnier .boa noite sou de Morro Reuter Rio Grande do sul.
Comentário: Hoje assiste,e eu estou assistindo. PATROCINIO MG
Comentário: Eu sou Maria quero envelhecer costurando pois eu amo costura agora assistindo você estou costurando máscaras.
Comentário: eu cheguei na metade da live, amei, e agora estou vendo de novo pra acompanhar o início. amei o depoimento da cmc, lindo a história dela.
Comentário:

Comentário: Maravilhosa aula!
Comentário: to assitindo agora porque nao consigo assistir ao vivo porque estou no trabalho
Comentário: Maravilhosa que lindos arrasou
Comentário: Blumenau -SC
Comentário: Mas não  estou conseguindo acompanhar eu estou sem máquina
Comentário: Lorena Santiago de Bom jesus do galho. MG
Comentário: Diana ,vc é  maravilhosa ensina da dicas,com carinho que se torna mais fácil pra nós aprendiz. Quero aprender como cortar seda.
Comentário: Boa noite Diana! Estou amando a semana CMC! É a minha primeira vez e vou fazer as peças sim!!!
Comentário: Interessante top 👍❤️❤️
Comentário: Deus abençoe você e sua família de Cmcs e essa equipe maravilhosa que você tem.🤗😘❤️no coração de todos.🙏
Comentário: Muito dica boa, vc passa a informacao de modo bem pratico, e facil de entender. Amando te assistir.
Alba Mara, Linhares ES.
Comentário: Eu costuro mas preciso aprender mais.eu sou Maria Lima Santos Silva Souza.de Ilhéus Bahia.a costura me ajudou na depressao
Comentário: Que

...

Autor: Delza Pereira
Autor: Loreni Machado de Lima
Autor: Cecília Santos
Autor: Denise Santos
Autor: Lia Mara
Autor: Mary Fontenelle
Autor: Zelia Maria
Autor: Sandra Markus de Oliveira
Autor: Rita LIBERATO
Autor: Daisy Salvador de Oliveira da Silva
Autor: Luciana Alfarth Gehrke
Autor: Ivanete Abraão do nascimento
Autor: Olinda Damaceno
Autor: Silvania Mendes
Autor: Rosilene Barreto
Autor: Rosemairy teixeira de holanda
Autor: Cristina Borges De Oliveira
Autor: Lenita Duvorak
Autor: Dilvaney Gomes
Autor: Artes com a Eloá
Autor: Erisangela Lopis
Autor: Reni Wetsphal
Autor: Sonia Mello
Autor: Eny Garcia da Silva Mazucato
Autor: Maria Inês da Silva Higuchi
Autor: Rosario Maria
Autor: Antonia Ribeiro
Autor: Marlene Guedes
Autor: Vanice Gelesky
Autor: Cheles costa
Autor: Elaine Fonseca
Autor: diana sousa
Autor: Mini mundo Isabel
Autor: Carmelita Maria
Autor: Ester Miranda Alves
Autor: Adinair Vilas boas
Autor: silsa velasques
Autor: Elisângela Morena
Autor: Graça Dutra
Autor: Ana Carvalho
Aut

Autor: Maria Moreira
Autor: ROSENILDA FREIRE FREIRE
Autor: Luizafranciscadesousaalencar Alencar
Autor: Telma Alves
Autor: Cláudia Sampaio
Autor: Ana Campos
Autor: Magnolia Costa
Autor: Lucinara Silveira De Lima
Autor: Arlete Alves jacobsen
Autor: Barbara Borba
Autor: Cheles costa
Autor: Elizabete Silva
Autor: Carmelina Alves
Autor: Maria Elizabeth
Autor: Maria Moura
Autor: Dela joverci Maria Ferreira
Autor: Dapaz Santos
Autor: Nilsa Souza lima
Autor: Rejane Alves Dos Santos
Autor: Luh Moreira
Autor: Ludimira Rocha
Autor: Berchiolina Reis
Autor: Elizabete Pinto de Oliveira
Autor: Ester Miranda Alves
Autor: Luciana Santos
Autor: Marta Da Silva
Autor: Cristina Borges De Oliveira
Autor: Ozita Oliveira
Autor: Hercilia Alves
Autor: Ivone Alves
Autor: Eleni Costa Andrade
Autor: vera costura
Autor: Linda Gomes
Autor: Vanda Monteiro
Autor: Nalva Santos
Autor: Maria Elizabeth
Autor: Berchiolina Reis
Autor: Márcia Bento da rocha medeiros
Autor: Gal Pil
Autor: Marcia Maria Oliveira Barbosa Barbosa

Autor: Maria de Lourdes Afonso de Arruda
Autor: Maria Jose pazini Cardoso
Autor: Odete Hillebrand
Autor: Lenir Francisca
Autor: Maria Silva
Autor: Vilma Rocha
Autor: Marina Souza
Autor: Hilda Barreto
Autor: Edinalva Dina
Autor: e suas brincadeiras malukas com vocês
Autor: Carmem Lucia
Autor: Maria Milagres
Autor: Roselandia Lima Da Silva Lima Da Silva
Autor: Eliane Souza
Autor: cristina batista
Autor: Marcilene Cardoso da Silva
Autor: Ilda correia Lima
Autor: Irenildes SouzaSantos
Autor: vitinho power
Autor: Luzia Gomes
Autor: Isabel Cristina Oliveira Neiva
Autor: Amaro Januario
Autor: Marcia Avelino
Autor: Valda Oliveira
Autor: Sônia Lima
Autor: Telma de jesus
Autor: Elisa Helena Bauab
Autor: Edivalda Rodrigues
Autor: Irenildes SouzaSantos
Autor: Marli Turetta
Autor: Irani Santos da Silva
Autor: noely souza
Autor: Ieda Moraes
Autor: Alaide Mendonça
Autor: May N
Autor: Maria Ines Da Silva Miranda
Autor: Vivi Verdelly
Autor: Maria Araújo
Autor: vitinho power
Autor: Bete Santana
Autor: L

Autor: Sandra Teófilo Ribeiro
Autor: Erli Oliveira
Autor: nadir lousada
Autor: Evanilde Santos Lima
Autor: Maria Oliveira
Autor: Maria Balbino
Autor: lindiomar lopes pinto
Autor: Elizabeth Nilza
Autor: Elisabete Ribeiro
Autor: Lenir Paiva
Autor: noeli bordignon
Autor: Rita Ferreira
Autor: Aurenice Amaral
Autor: noeli bordignon
Autor: Lidiane pereira Murbak
Autor: Vania Rocha
Autor: vitinho power
Autor: Maria jose Pantoja
Autor: Janderson Auzier Moraes
Autor: Elaine Regina Ribeiro silva
Autor: Adriana Dos Santos Silva
Autor: Lenir Paiva
Autor: Vilma Santos
Autor: Vanessa Claudia
Autor: Prô Ju
Autor: Gildete Matos
Autor: Maria Inácio Pinheiro
Autor: Meire An já fhffhdfk
Autor: Eloisa S.tonetto
Autor: Jandira Ferreira
Autor: Ivone ak Lima
Autor: Greiciane Lemos
Autor: Josi Rocha
Autor: saide beatriz
Autor: iolandinha pocoto
Autor: cristina batista
Autor: Celma Santana
Autor: Otacilia Maria
Autor: Isaque Costa
Autor: MISS
Autor: Maria Oliveira
Autor: Maria Oliveira
Autor: Maria Liberdade
A

ValueError: Length of values (1580) does not match length of index (701)